# IO Lab

In [1]:
import os
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
from appinsight.utils.convert import ToSpark
from appinsight.infrastructure.frameworks.spark.factory import SparkSessionFactory
from appinsight.utils.file import IOService
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 20


In [2]:
fp1 = "data/reviews.pkl"
fp2 = "data/reviews"
df1 = IOService.read(fp1)
df1.info()
IOService.write(filepath=fp2, data=df1, partition_cols = ["category"])
df2 = IOService.read(fp2)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22166591 entries, 0 to 22166590
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           string        
 1   app_id       string        
 2   app_name     string        
 3   category_id  object        
 4   category     object        
 5   author       object        
 6   rating       float64       
 7   content      string        
 8   vote_sum     Int64         
 9   vote_count   Int64         
 10  date         datetime64[ns]
dtypes: Int64(2), datetime64[ns](1), float64(1), object(3), string(4)
memory usage: 1.9+ GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22166591 entries, 0 to 22166590
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           string        
 1   app_id       string        
 2   app_name     string        
 3   category_id  object        
 4   author       object        
 5   rating       

In [ ]:
IOService.write(data=df1, filepath=fp2)

In [ ]:
df1.head()
df1.info()

In [ ]:
factory = SparkSessionFactory()
spark = factory.build()
df3 = spark.read.parquet(fp2)
df3.printSchema()


In [ ]:
df3.show()